# CPSC529: 04_DataPreparation_3_Transformer_Pipeline

Source: https://www.analyticsvidhya.com/blog/2021/05/understanding-column-transformer-and-machine-learning-pipelines/

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')


## 0. Data Loading

In [2]:
df = pd.read_csv(os.path.join('data', 'titanic.csv'))

## 1. Train Test Split

In [3]:
#drop the non-required columns
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

X, y = df.drop(columns=['Survived']), df['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print(type(X), type(y), type(X_train), type(y_train))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


In [4]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [6]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 2. Create Transformers

In [7]:
#1st Imputation Transformer
trf1 = ColumnTransformer([
        ('impute_age',SimpleImputer(),["Age"]),
        ('impute_embarked',SimpleImputer(strategy='most_frequent'),["Embarked"])
    ],remainder='passthrough') #,remainder='passthrough'

In [8]:
# print(type(X_train), type(trf1.fit_transform(X_train)),'\n')
print(X_train.head(), '\n\n', trf1.fit_transform(X_train)[:5])

     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
331       1    male  45.5      0      0  28.5000        S
733       2    male  23.0      0      0  13.0000        S
382       3    male  32.0      0      0   7.9250        S
704       3    male  26.0      1      0   7.8542        S
813       3  female   6.0      4      2  31.2750        S 

 [[45.5 'S' 1 'male' 0 0 28.5]
 [23.0 'S' 2 'male' 0 0 13.0]
 [32.0 'S' 3 'male' 0 0 7.925]
 [26.0 'S' 3 'male' 1 0 7.8542]
 [6.0 'S' 3 'female' 4 2 31.275]]


In [9]:
trf1.__dict__

{'transformers': [('impute_age', SimpleImputer(), ['Age']),
  ('impute_embarked', SimpleImputer(strategy='most_frequent'), ['Embarked'])],
 'remainder': 'passthrough',
 'sparse_threshold': 0.3,
 'n_jobs': None,
 'transformer_weights': None,
 'verbose': False,
 'verbose_feature_names_out': True,
 'feature_names_in_': array(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'],
       dtype=object),
 'n_features_in_': 7,
 '_columns': [['Age'], ['Embarked']],
 '_transformer_to_input_indices': {'impute_age': [2],
  'impute_embarked': [6],
  'remainder': [0, 1, 3, 4, 5]},
 '_n_features': 7,
 '_remainder': ('remainder', 'passthrough', [0, 1, 3, 4, 5]),
 'sparse_output_': False,
 '_name_to_fitted_passthrough': {'remainder': FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one')},
 'transformers_': [('impute_age', SimpleImputer(), ['Age']),
  ('impute_embarked', SimpleImputer(strategy='most_frequent'), ['Embarked']),
  ('rema

In [10]:
trf1.verbose_feature_names_out

True

In [11]:
#2nd One Hot Encoding ['Sex', 'Embarked']
trf2 = ColumnTransformer([
        ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'),[1,3])
    ], remainder='passthrough')

In [12]:
trf2.__dict__

{'transformers': [('ohe_sex_embarked',
   OneHotEncoder(handle_unknown='ignore', sparse=False),
   [1, 3])],
 'remainder': 'passthrough',
 'sparse_threshold': 0.3,
 'n_jobs': None,
 'transformer_weights': None,
 'verbose': False,
 'verbose_feature_names_out': True}

In [13]:
#3rd Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [14]:
#4th Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [15]:
#5th Model
trf5 = DecisionTreeClassifier()

## 3. Create Pipeline

### 3.1. Pipeline with model excluded

In [16]:
pipe_without_model = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4)
])

# Display Pipeline
from sklearn import set_config
set_config(display='diagram')

In [37]:
X_transformed=pipe_without_model.fit_transform(X, y)

print(pipe_without_model.steps[0][1].get_feature_names_out(),"\n")
print(pipe_without_model.steps[1][1].get_feature_names_out(),"\n")
print(pipe_without_model.steps[2][1].get_feature_names_out(),"\n")

print(X_transformed)

['impute_age__Age' 'impute_embarked__Embarked' 'remainder__Pclass'
 'remainder__Sex' 'remainder__SibSp' 'remainder__Parch' 'remainder__Fare'] 

['ohe_sex_embarked__x1_C' 'ohe_sex_embarked__x1_Q'
 'ohe_sex_embarked__x1_S' 'ohe_sex_embarked__x3_female'
 'ohe_sex_embarked__x3_male' 'remainder__x0' 'remainder__x2'
 'remainder__x4' 'remainder__x5' 'remainder__x6'] 

['scale__x0' 'scale__x1' 'scale__x2' 'scale__x3' 'scale__x4' 'scale__x5'
 'scale__x6' 'scale__x7' 'scale__x8' 'scale__x9'] 

[[0.         1.         0.         ... 0.125      0.         0.01415106]
 [1.         0.         1.         ... 0.125      0.         0.13913574]
 [0.         1.         1.         ... 0.         0.         0.01546857]
 ...
 [0.         1.         1.         ... 0.125      0.33333333 0.04577135]
 [1.         0.         0.         ... 0.         0.         0.0585561 ]
 [0.         0.         0.         ... 0.         0.         0.01512699]]


### 3.2. Pipeline with model included

In [18]:
pipe_with_model = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [19]:
pipe_with_model.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Embarked'])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002039B7D95A0>)),
                ('trf5', DecisionTreeClassifier())])

## 4. Model Evaluation

In [20]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe_with_model, X_train, y_train, cv=5, scoring="accuracy").mean()

0.7852654387865655

In [21]:
from sklearn.metrics import accuracy_score

# find accuracy scores
accuracy = accuracy_score(y_test, pipe_with_model.predict(X_test))
print("The accuracy of prediction is: ", accuracy)

The accuracy of prediction is:  0.7932960893854749
